In [52]:
!pip install utils pandas natsort nltk wandb tensorboard

In [67]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd
import wandb
import unicodedata
from collections import Counter

In [54]:
cudnn.benchmark = True
cudnn.deterministic = False

In [71]:
def clean_text(text):
    # Нормализация до NFC
    text = unicodedata.normalize('NFC', text)
    
    # Удаление всех контролирующих символов (категория C*)
    text = ''.join(c for c in text if not unicodedata.category(c).startswith('C'))
    
    # Разрешённые комбинирующие символы
    allowed_combining = {'\u0301'}  # Только острый акцент
    text = ''.join(c for c in text if not unicodedata.combining(c) or c in allowed_combining)
    
    # Явное удаление специфических нежелательных символов
    text = text.replace('\u200e', '').replace('\u200f', '')  # Удаление LRM и RLM
    
    return text

In [78]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    
    if opt.lang_char == 'None':
        characters = ''
        total_samples_before = 0
        total_samples_after = 0
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep=',', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            print(f"Прочитано {len(df)} примеров из {csv_path}")
            df['words'] = df['words'].apply(clean_text)
            print(f"Пример до очистки: {df['words'].iloc[0]}")
            # Удаление примеров с пустыми метками после очистки
            df = df[df['words'].str.strip() != '']
            print(f"Оставлено {len(df)} примеров после очистки из {csv_path}")
            total_samples_before += len(df)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character = ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    
    print(f"Общее количество символов в opt.character: {len(opt.character)}")
    
    # Дополнительная проверка на наличие неподдерживаемых символов
    unsupported_chars = set()
    for data in opt['select_data'].split('-'):
        csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
        df = pd.read_csv(csv_path, sep=',', engine='python', usecols=['filename', 'words'], keep_default_na=False)
        df['words'] = df['words'].apply(clean_text)
        for word in df['words']:
            for char in word:
                if char not in opt.character:
                    unsupported_chars.add(char)

        print(df.head())
    if unsupported_chars:
        print(f"Найдено неподдерживаемых символов: {unsupported_chars}")
    else:
        print("Неподдерживаемых символов не найдено.")
    
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [129]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(
                csv_path,
                sep='^([^,]+),',  # Обратите внимание на корректность разделителя
                engine='python',
                usecols=['filename', 'words'],
                keep_default_na=False
            )
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
            
            # Вывод первых строк датасета
            print(f"Первые строки из {csv_path}:")
            print(df.head())
        
        characters = sorted(set(characters))
        opt.character = ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [111]:
class AttrDict(dict):
    """A simple class to convert dictionary keys to attributes."""
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [121]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    
    print(f"Значение opt.lang_char: {opt.lang_char}")
    print(f"Значение opt.select_data: {opt.select_data}")
    print(f"Распарсенное opt.select_data.split('-'): {opt.select_data.split('-')}")
    
    # Инициализация переменных для подсчёта
    total_comma_sentences = 0
    characters = ''
    
    # Обработка select_data
    for data in opt.select_data.split('-'):
        csv_path = os.path.join(opt.train_data, data, 'labels.csv')
        print(f"\nПроверка файла: {csv_path}")  # Отладочный вывод
        
        # Проверка существования файла
        if not os.path.exists(csv_path):
            print(f"Файл {csv_path} не существует.")
            continue
        
        try:
            df = pd.read_csv(
                csv_path, 
                sep=',', 
                engine='python', 
                usecols=['filename', 'words'], 
                keep_default_na=False
            )
            print(f"Прочитано {len(df)} примеров из {csv_path}")
        except Exception as e:
            print(f"Ошибка чтения файла {csv_path}: {e}")
            continue
        
        # Подсчёт предложений с запятыми
        comma_sentences = df['words'].str.contains(',').sum()
        total_comma_sentences += comma_sentences
        print(f"Количество предложений с запятыми в {csv_path}: {comma_sentences}")
        
        # Собираем символы из всех предложений
        all_char = ''.join(df['words'])
        characters += ''.join(set(all_char))
    
    # Общий вывод по запятым
    print(f"\nОбщее количество предложений с запятыми: {total_comma_sentences}")
    
    # Обработка opt.character в зависимости от lang_char
    if opt.lang_char == 'None':
        # Если lang_char равен строке 'None', формируем символы из данных
        characters = sorted(set(characters))
        opt.character = ''.join(characters)
    else:
        # Если lang_char содержит символы, комбинируем их с number и symbol
        opt.character = opt.number + opt.symbol + opt.lang_char
        print("opt.lang_char не равен 'None', комбинируем символы из number, symbol и lang_char.")
    
    print(f"\nОбщее количество символов в opt.character: {len(opt.character)}")
    print(f"Содержимое opt.character: {opt.character}")
    print(f"Включает ли opt.character запятую: {',' in opt.character}")
    
    # Дополнительная проверка на наличие неподдерживаемых символов
    unsupported_chars = set()
    for data in opt.select_data.split('-'):
        csv_path = os.path.join(opt.train_data, data, 'labels.csv')
        print(f"\nПроверка неподдерживаемых символов в {csv_path}")
        
        # Проверка существования файла
        if not os.path.exists(csv_path):
            print(f"Файл {csv_path} не существует.")
            continue
        
        try:
            df = pd.read_csv(
                csv_path, 
                sep=',', 
                engine='python', 
                usecols=['filename', 'words'], 
                keep_default_na=False
            )
        except Exception as e:
            print(f"Ошибка чтения файла {csv_path}: {e}")
            continue
        
        for word in df['words']:
            for char in word:
                if char not in opt.character:
                    unsupported_chars.add(char)
    
    if unsupported_chars:
        print(f"\nНайдено неподдерживаемых символов: {unsupported_chars}")
    else:
        print("\nНеподдерживаемых символов не найдено.")
    
    # Создание директории для сохранения моделей
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [128]:
opt = get_config("config_files/sah_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/
opt.select_data: ['sah_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/	 dataset: sah_filtered
all_data//sah_filtered
sub-directory:	/sah_filtered	 num samples: 60524
num total samples of sah_filtered: 60524 x 1.0 (total_data_usage_ratio) = 60524
num samples of sah_filtered per batch: 16 x 1.0 (batch_ratio) = 16
--------------------------------------------------------------------------------
Total_batch_size: 16 = 16
--------------------------------------------------------------------------------
dataset_root:    all_data/val	 dataset: /
all_data/val/
sub-directory:	/.	 num samples: 15132
---------------------------------------------------------------

KeyError: '"'

In [130]:
def initialize_wandb(opt):
    wandb.init(
        project="EasyOCR_Yakut",      # Название вашего проекта в wandb
        config=opt,                   # Передаём конфигурацию как гиперпараметры
        name=opt.experiment_name,     # Название запуска
        sync_tensorboard=True         # Если используете TensorBoard
    )
    # Логирование дополнительных параметров при необходимости
    wandb.config.update(opt)

In [131]:
opt = get_config("config_files/sah_filtered_config.yaml")
initialize_wandb(opt)

In [132]:
# Запуск процесса обучения и передаём wandb.run в функцию train
train(opt, amp=False, wandb_run=wandb)

# Завершаем сессию wandb
wandb.finish()

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/
opt.select_data: ['sah_filtered']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/	 dataset: sah_filtered
all_data//sah_filtered
sub-directory:	/sah_filtered	 num samples: 60524
num total samples of sah_filtered: 60524 x 1.0 (total_data_usage_ratio) = 60524
num samples of sah_filtered per batch: 16 x 1.0 (batch_ratio) = 16
--------------------------------------------------------------------------------
Total_batch_size: 16 = 16
--------------------------------------------------------------------------------
dataset_root:    all_data/val	 dataset: /
all_data/val/
sub-directory:	/.	 num samples: 15132
---------------------------------------------------------------

SystemExit: 0

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
